In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_name: str
    local_data_file: Path

In [ ]:
from transformerEnFa.constants import *
from transformerEnFa.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            dataset_name=config.dataset_name,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [ ]:
import os
from datasets import load_dataset
from transformerEnFa.logging import logger
from transformerEnFa.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            dataset = load_dataset(self.config.dataset_name)
            os.makedirs(self.config.local_data_file, exist_ok=True)
            # since the dataset only has train
            dataset['train'].to_csv(os.path.join(self.config.local_data_file, "ds_raw.csv"))
            logger.info(f"{self.config.dataset_name} download!")
        else:
            file_path = Path(self.config.local_data_file) / 'ds_raw.csv'
            logger.info(f"File already exists of size: {get_size(file_path)}")
           


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e 
